## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [62]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")

#check the first five rows
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bekhtery,UA,46.2510,32.2927,27.84,75,12,7.02,few clouds
1,1,Tiksi,RU,71.6872,128.8694,-7.28,97,100,8.19,overcast clouds
2,2,Guapiacu,BR,-20.7950,-49.2203,79.09,69,0,3.44,clear sky
3,3,Albany,US,42.6001,-73.9662,20.68,70,89,2.68,overcast clouds
4,4,Bocaiuva,BR,-17.1078,-43.8150,69.51,91,15,4.81,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Guapiacu,BR,-20.7950,-49.2203,79.09,69,0,3.44,clear sky
8,8,Pemangkat,ID,1.1667,108.9667,82.72,75,24,7.11,few clouds
12,12,East London,ZA,-33.0153,27.9116,71.85,66,0,18.59,clear sky
14,14,Puerto Ayora,EC,-0.7393,-90.3518,72.86,90,86,4.79,overcast clouds
15,15,Avarua,CK,-21.2078,-159.7750,78.85,94,75,21.85,moderate rain
16,16,Hithadhoo,MV,-0.6000,73.0833,82.62,69,66,16.49,broken clouds
19,19,Rikitea,PF,-23.1203,-134.9692,78.71,72,100,11.59,overcast clouds
20,20,Trinidad,BO,-14.8333,-64.9000,72.28,98,85,3.74,overcast clouds
22,22,Bredasdorp,ZA,-34.5322,20.0403,78.28,88,40,4.61,scattered clouds
23,23,Tual,ID,-5.6667,132.7500,82.78,78,90,14.12,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                214
City                   214
Country                212
Lat                    214
Lng                    214
Max Temp               214
Humidity               214
Cloudiness             214
Wind Speed             214
Current Description    214
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

# recheck to ensure empty rows were dropped
clean_df.count()

City_ID                212
City                   212
Country                212
Lat                    212
Lng                    212
Max Temp               212
Humidity               212
Cloudiness             212
Wind Speed             212
Current Description    212
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Guapiacu,BR,79.09,clear sky,-20.7950,-49.2203,
8,Pemangkat,ID,82.72,few clouds,1.1667,108.9667,
12,East London,ZA,71.85,clear sky,-33.0153,27.9116,
14,Puerto Ayora,EC,72.86,overcast clouds,-0.7393,-90.3518,
15,Avarua,CK,78.85,moderate rain,-21.2078,-159.7750,
16,Hithadhoo,MV,82.62,broken clouds,-0.6000,73.0833,
19,Rikitea,PF,78.71,overcast clouds,-23.1203,-134.9692,
20,Trinidad,BO,72.28,overcast clouds,-14.8333,-64.9000,
22,Bredasdorp,ZA,78.28,scattered clouds,-34.5322,20.0403,
23,Tual,ID,82.78,overcast clouds,-5.6667,132.7500,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()


    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -22.530817, 'lng': -52.1721372}, 'viewport': {'northeast': {'lat': -22.5295264697085, 'lng': -52.1707586697085}, 'southwest': {'lat': -22.5322244302915, 'lng': -52.1734566302915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'icon_background_color': '#909CE1', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet', 'name': 'MINI HOTEL', 'photos': [{'height': 1658, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/104115595542688894372">Ju Oliveira</a>'], 'photo_reference': 'Aap_uEBmsExvX0iYWLg0gqJuGb_w

In [38]:
# import numpy dependency to help with cleaning data without a hotel name
import numpy as np

In [46]:
# 7. Drop the rows where there is no Hotel Name --> for help with how to replace a blank cell, I looked answers on the following site: https://stackoverflow.com/questions/13445241/replacing-blank-values-white-space-with-nan-in-pandas
# replace blank cells with NaN
hotel_df = hotel_df.replace(r'^\s*$', np.nan, regex=True)

# drop rows containing NaN
clean_hotel_df = hotel_df.dropna()

# # review DataFrame to confirm it looks right before exporting to CSV
clean_hotel_df.tail(20)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
642,Pemba,MZ,77.70,broken clouds,-12.9740,40.5178,Guesthouse Samotina
649,Santa Cruz,BO,74.66,scattered clouds,-17.8000,-63.1667,Aviador Hotel Boutique
652,Iracoubo,GF,74.89,overcast clouds,5.4802,-53.2011,Le Gîte d’Iracoubo
658,Praia,CV,72.14,scattered clouds,14.9215,-23.5087,LT Aparthotel
663,Urucara,BR,73.92,heavy intensity rain,-2.5364,-57.7600,Pousada Maria Bonita
664,Manzanillo,CU,73.18,clear sky,20.3433,-77.1167,"HOSTAL ""THE YELLOW HOUSE"""
668,Pathein,MM,83.98,few clouds,16.7833,94.7333,New Pammawaddy Hotel
669,Mahayag,PH,84.36,broken clouds,8.1183,123.4455,Eriberta Spring
672,Salcedo,PH,82.42,few clouds,12.8105,121.4674,Yamaji Freeman Farm
675,Binabalian,PH,80.53,broken clouds,16.4202,119.9279,Sundowners Vacation Villas


In [47]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [51]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [63]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))